### Главы 9 - 10

Усовершенствование сети для MNIST новым функциями активации

In [2]:
import mnist
import numpy as np, sys
np.random.seed(1) # фиксируем псевдослучайный генератор случайных чисел

x_train = mnist.train_images()[:1000,:,:]
y_train = mnist.train_labels()[:1000,...]

x_test = mnist.test_images()[:2000,:,:]
y_test = mnist.test_labels()[:2000,...]

In [9]:
images, labels = x_train[:1000].reshape(1000, 28*28)/255 , y_train[:1000] # загружаем 1000 изображений и меток (train)
one_hot_labels = np.zeros((1000, 10)) # готовим нулевую матрицу. По строкам обучающие примеры, по столбцам метка
for n,l in enumerate(labels):
    one_hot_labels[n,l] = 1 # для каждой строки в одном из столбцов ставим 1, соответствующую метке (train)
labels = one_hot_labels # переопределяем labels

test_images, test_labels = x_test.reshape(-1, 28*28)/255, y_test # загружаем 1000 изображений и меток (test)
one_hot_labels_test = np.zeros((x_test.shape[0], 10)) # готовим нулевую матрицу. По строкам обучающие примеры, по столбцам метка
for n, l in enumerate(test_labels): # для каждой строки в одном из столбцов ставим 1, соответствующую метке (test)
    one_hot_labels_test[n,l] = 1
test_labels = one_hot_labels_test


tanh = lambda x: np.tanh(x)
tanh2deriv = lambda output: 1 - (output ** 2)
softmax = lambda x: np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)
    
# relu = lambda x: (x>=0)*x # пишем лямбда-функцию relu (обнуляет отрицательные веса)
# relu2deriv = lambda x: x >= 0 # пишем лямбда-функцию relu2deriv (обнуляет отрицательные нейроны)

# устанавливаем параметры
alpha, iterations, hidden_size, pixels_per_image, num_labels = (2, 301, 100, 784, 10)
batch_size = 100

# инициализируем случайные веса
weights_0_1 = 0.02*np.random.random((pixels_per_image, hidden_size)) - 0.01 # для слоя 1, форма (784, 100)
weights_1_2 = 0.2*np.random.random((hidden_size, num_labels)) - 0.1 # для слоя 2, форма (100, 10)

for j in range(iterations): # итерируемся
#     error, correct_cnt = (0.0, 0) # error - RSE, correct_cnt - количество True-Positive ответов
    correct_cnt = 0
    for i in range(round(images.shape[0]/batch_size)): # бежим по обучающим примерам
        batch_start, batch_end = ((i * batch_size),((i+1) * batch_size)) # 0-99, 100-199 ...
        layer_0 = images[batch_start:batch_end,:].reshape(batch_size,pixels_per_image) # подаем на входной слой изображение (100,784)
        layer_1 = tanh(layer_0.dot(weights_0_1)).reshape(batch_size,batch_size) # скалярное перемножение входного слоя (100,784) на веса (784,100), 
                                                # получаем (100,100), затем применяем relu для обнуления нейронов с отриц. знач.
        dropout_mask = np.random.randint(2, size=layer_1.shape) # создаем маску вида array([[0, 1, 0, ... 0, 0, 1]]),
                                                                # где вероятность единиц и нулей по 50% (100,100)
        layer_1 *= dropout_mask * 2 # применяем маску, но умноженную на 2 
                        # (т.к. сигнал от слоя 1 примерно в 2 раза уменьшается после применения маски, поэтому его нужно усилить)
        
        layer_2 = softmax(layer_1.dot(weights_1_2).reshape(batch_size,num_labels)) # скалярное перемножение внутреннего слоя (100,100) на веса (100,10), 
                                            # получаем выходной слой (100,10) с вероятностями каждого лейбла.
        
#         error += np.sum((layer_2 - labels[batch_start:batch_end,:])**2) # учитываем ошибку по i-тому обучающему примеру 
#                                                         # в общей ошибке для j-той итерации
#         print(error)
        for k in range(batch_size): # итерируемся по примерам в батче
            correct_cnt += int(np.argmax(layer_2[k,:]) == np.argmax(labels[batch_start+k,:])) # учитываем True-Positive ответ (если есть)
                                                                                # в общем показателей correct_cnt для j-той итерации
        layer_2_delta = (layer_2 - labels[batch_start:batch_end,:])/\
                                            (batch_size * layer_2.shape[0]) # считаем отклонение по выходному слою (100,10)
        #print(layer_2_delta.shape)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1) # считаем отклонение по среднему слою:
                                                        # скалярно перемножаем отклонение по выходному слою (100,10)
                                                        # на транспонированную матрицу весов weights_1_2 (10,100)
                                                        # получаем вектор (100,100)
        layer_1_delta *= dropout_mask # при обратном распространении корректировать обнуленные маской нейроны не следует
                                        # поэтому той же маской обнуляем полученные дельты

        weights_1_2 -= alpha * layer_1.T.dot(layer_2_delta) # корректируем веса для выходного слоя: 
                                                            # из матрицы текущих весов (100,10) вычитаем
                                                            # помноженный на скорость обучения слой 1 транспонированный (100,100)
                                                            # скалярно умножаем на отклонение по слою 2 (100,10)
                                                            # получаем (100,10) - такой же формы, как и были наши веса weights_1_2
        weights_0_1 -= alpha * layer_0.T.dot(layer_1_delta) # корректируем веса для скрытого слоя: 
                                                            # из матрицы текущих весов (784, 100) вычитаем
                                                            # помноженный на скорость обучения слой 0 транспонированный (784,100)
                                                            # скалярно умножаем на отклонение по слою 1 (100,100)

                                                            # получаем (784,100) - такой же формы, как и были наши веса weights_0_1
        
    test_correct_cnt = 0 # correct_cnt - количество True-Positive ответов

    for i in range(test_images.shape[0]): # бежим по обучающим примерам
        layer_0 = test_images[i,:].reshape(1,-1) # подаем на входной слой изображение (1,784)
        layer_1 = tanh(layer_0.dot(weights_0_1)).reshape(1,-1) # скалярное перемножение входного слоя (1,784) на веса (784,40), 
                                                # получаем (1,40), затем применяем relu для обнуления нейронов с отриц. знач.
        layer_2 = layer_1.dot(weights_1_2).reshape(1,-1) # скалярное перемножение внутреннего слоя (1,40) на веса (40,10), 
                                            # получаем выходной слой (1,10) с вероятностями каждого лейбла.
        
        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i,:])) # учитываем True-Positive ответ (если есть)
                                                                            # в общем показателей correct_cnt для j-той итерации
    if j % 10 == 0:
        print("Iter:" + str(j) +\
             " Test_Accuracy: " + str(test_correct_cnt/float(test_images.shape[0])) +\
              " Train_Accuracy: " + str(correct_cnt/float(images.shape[0])) )

Iter:0 Test_Accuracy: 0.323 Train_Accuracy: 0.206
Iter:10 Test_Accuracy: 0.6395 Train_Accuracy: 0.72
Iter:20 Test_Accuracy: 0.6615 Train_Accuracy: 0.748
Iter:30 Test_Accuracy: 0.69 Train_Accuracy: 0.782
Iter:40 Test_Accuracy: 0.7155 Train_Accuracy: 0.802
Iter:50 Test_Accuracy: 0.7385 Train_Accuracy: 0.835
Iter:60 Test_Accuracy: 0.753 Train_Accuracy: 0.849
Iter:70 Test_Accuracy: 0.768 Train_Accuracy: 0.869
Iter:80 Test_Accuracy: 0.7795 Train_Accuracy: 0.879
Iter:90 Test_Accuracy: 0.7865 Train_Accuracy: 0.881
Iter:100 Test_Accuracy: 0.792 Train_Accuracy: 0.888
Iter:110 Test_Accuracy: 0.7985 Train_Accuracy: 0.898
Iter:120 Test_Accuracy: 0.8005 Train_Accuracy: 0.898
Iter:130 Test_Accuracy: 0.8085 Train_Accuracy: 0.901
Iter:140 Test_Accuracy: 0.808 Train_Accuracy: 0.905
Iter:150 Test_Accuracy: 0.816 Train_Accuracy: 0.92
Iter:160 Test_Accuracy: 0.818 Train_Accuracy: 0.925
Iter:170 Test_Accuracy: 0.8205 Train_Accuracy: 0.917
Iter:180 Test_Accuracy: 0.8225 Train_Accuracy: 0.921
Iter:190 Test_A

### Сверточная сеть

In [112]:
def get_image_section(layer, row_from, row_to, col_from, col_to):
    '''Функция выбирает подобласть в изображении'''
    sub_section = layer[:, row_from:row_to, col_from:col_to]
    return sub_section.reshape(-1, 1, row_to - row_from, col_to - col_from)

In [113]:
aa = np.arange(64).reshape(8,8)
aa

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55],
       [56, 57, 58, 59, 60, 61, 62, 63]])

In [114]:
aa = np.array([aa]*3)
aa.shape

(3, 8, 8)

In [115]:
get_image_section(aa, 0, 2, 0, 2)

array([[[[0, 1],
         [8, 9]]],


       [[[0, 1],
         [8, 9]]],


       [[[0, 1],
         [8, 9]]]])

In [116]:
get_image_section(aa, 0, 2, 0, 2).shape

(3, 1, 2, 2)

In [119]:
# проверим функцию
layer_0 = images[batch_start:batch_end] # (100, 784)
layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28) # (100, 28, 28)

sects = []
kernel_rows, kernel_cols = (3, 3)

for row_start in range(layer_0.shape[1] - kernel_rows + 1): # бежим по строкам от 0 до 28-3 включительно
    for col_start in range(layer_0.shape[2] - kernel_cols + 1): # бежим по столбцам от 0 до 28-3 включительно
        sect = get_image_section(layer_0,
                                row_start, # будет перебираться от 0 до 25 включительно
                                row_start + kernel_rows, # будет перебираться от 3 до 28 включительно
                                col_start, # будет перебираться от 0 до 25 включительно
                                col_start + kernel_cols) # будет перебираться от 3 до 28 включительно
        sects.append(sect) # всего д.б. 26*26 = 676 массивов размера (100, 1, 3, 3)
print(f'sects {len(sects)}')       
expanded_input = np.concatenate(sects, axis=1) # соединяем массивы горизонтально (присоединяем справа) получаем (100, 676, 3, 3)
print(f'expanded_input {expanded_input.shape}')
es = expanded_input.shape # (100, 676, 3, 3) большой столб, составленный из столбиков 3 на 3
print(f'es {es}')
flattened_input = expanded_input.reshape(es[0]*es[1], -1) # каждый "плитку" 3 на 3 вытягиваем в строку, транспонируем и выстраиваем в ряд
# получаем (67600, 9)
print(f'flattened_input {flattened_input.shape}')

sects 676
expanded_input (104, 676, 3, 3)
es (104, 676, 3, 3)
flattened_input (70304, 9)


Полная реализация

In [120]:
num_kernels = 16 # сделаем 16 разных ядер с рандомными весами от 0 до 1 (которые входят в эти ядра)
kernels = np.random.random((kernel_rows*kernel_cols, num_kernels)) # делаем матрицу с весами для 16 ядер
kernels.shape

(9, 16)

In [122]:
kernels[0]

array([0.54395304, 0.77476367, 0.55578024, 0.68876649, 0.4469488 ,
       0.20733249, 0.38132192, 0.61622455, 0.8077445 , 0.30486368,
       0.97329327, 0.61889835, 0.44806278, 0.26944556, 0.97266354,
       0.3125513 ])

In [124]:
kernel_output = flattened_input.dot(kernels)
kernel_output.shape # тут получаем 16 прогнозов для каждой "плитки"

(70304, 16)

In [126]:
import mnist
import numpy as np, sys, time

np.random.seed(1) # фиксируем псевдослучайный генератор случайных чисел

x_train = mnist.train_images()[:1000,:,:]
y_train = mnist.train_labels()[:1000,...]

x_test = mnist.test_images()[:2000,:,:]
y_test = mnist.test_labels()[:2000,...]

In [155]:
images, labels = x_train[:1000].reshape(1000, 28*28)/255 , y_train[:1000] # загружаем 1000 изображений и меток (train)
one_hot_labels = np.zeros((1000, 10)) # готовим нулевую матрицу. По строкам обучающие примеры, по столбцам метка
for n,l in enumerate(labels):
    one_hot_labels[n,l] = 1 # для каждой строки в одном из столбцов ставим 1, соответствующую метке (train)
labels = one_hot_labels # переопределяем labels (1000, 10)

test_images, test_labels = x_test.reshape(-1, 28*28)/255, y_test # загружаем 1000 изображений и меток (test)
one_hot_labels_test = np.zeros((x_test.shape[0], 10)) # готовим нулевую матрицу. По строкам обучающие примеры, по столбцам метка
for n, l in enumerate(test_labels): # для каждой строки в одном из столбцов ставим 1, соответствующую метке (test)
    one_hot_labels_test[n,l] = 1
test_labels = one_hot_labels_test

tanh = lambda x: np.tanh(x)
tanh2deriv = lambda output: 1 - (output ** 2)
softmax = lambda x: np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

# устанавливаем параметры
alpha, iterations, pixels_per_image, num_labels = (2, 301, 784, 10)
batch_size = 100

input_rows, input_cols = (28, 28)
kernel_rows, kernel_cols = (3, 3)
num_kernels = 16

# в скрытом слое будут приниматься результаты 26*26 раз просмотров каждого изображения 16-ю ядрами
# то есть по каждому изображению в общей сложности будет 10816 прогнозов 
hidden_size = ((input_rows - kernel_rows+1) * (input_cols - kernel_cols+1)) * num_kernels # (28-3+1)*(28-3+1)*16 = 10816
kernels = 0.02 * np.random.random((kernel_rows*kernel_cols, num_kernels)) - 0.01 # 16 ядер со случайными 9-ю весами (9,16)
# причем веса в диапазоне от -0,01 до 0,01

# инициализируем случайные веса: для каждого из 10816 прогнозов с нейронов скрытого слоя готовим по 10 весов 
weights_1_2 = 0.2*np.random.random((hidden_size, num_labels)) - 0.1 # получаем (10816, 10) 
# причем диапазон весов от -0,1 до 0,1

def get_image_section(layer, row_from, row_to, col_from, col_to):
    '''Функция выбирает подобласть в изображении'''
    sub_section = layer[:, row_from:row_to, col_from:col_to]
    return sub_section.reshape(-1, 1, row_to - row_from, col_to - col_from)

glob_start_time = time.process_time()
for j in range(iterations): # итерируемся по количеству итераций
    start_time = time.process_time()
    correct_cnt = 0 # количество TP-прогнозов
    for i in range(round(images.shape[0]/batch_size)): # бежим по батчам из обучающих примеров
        batch_start, batch_end = ((i * batch_size),((i+1) * batch_size)) # 0-99, 100-199 ...
        layer_0 = images[batch_start:batch_end,:].reshape(-1,28, 28) # подаем на входной слой изображение (100,28,28) 
        # или (128,28,28) в зависимости от размера батча
        
        sects = [] # пустой список для отсмотренных плиточек 3*3
        for row_start in range(layer_0.shape[1] - kernel_rows + 1): # бежим по строкам от 0 до 28-3=25 включительно
                                                            # и смотрим интересующую область сразу по всем изображениям в батче 
            for col_start in range(layer_0.shape[2] - kernel_cols + 1): # бежим по столбцам от 0 до 28-3 включительно
                sect = get_image_section(layer_0, # вырезаем из одного и того же места всех изображений в батче квадрат
                                        row_start, # будет перебираться от 0 до 25 включительно
                                        row_start + kernel_rows, # будет перебираться от 3 до 28 включительно
                                        col_start, # будет перебираться от 0 до 25 включительно
                                        col_start + kernel_cols) # будет перебираться от 3 до 28 включительно
                                        #         print(f'row_start = {row_start} col_start = {col_start}')
                sects.append(sect) # всего д.б. 26*26 = 676 массивов-стопочек размера (128, 1, 3, 3)
        expanded_input = np.concatenate(sects, axis=1) # соединяем массивы горизонтально (присоединяем справа) 
        # получаем (128, 676, 3, 3) - "стену" высотой 128, толщиной 3 и протяженностью 676
        es = expanded_input.shape # (128, 676, 3, 3) просто форма
        flattened_input = expanded_input.reshape(es[0]*es[1], -1) # каждый "плитку" 3*3 вытягиваем в строку, 
        # выкладываем в штабелем и получаем (86528, 9)
        kernel_output = flattened_input.dot(kernels)  # получаем по 16 прогнозов для каждой "плитки"
                                                        # (86528, 9).dot(9,16) = (86528,16)
        layer_1 = tanh(kernel_output.reshape(es[0], -1)) # подаем полученные прогнозы на скрытый слой и сразу для
                                        # последующих умножений меняем форму на (128,10816) и применяем функцию активации,
                                        # которая загоняет полученные входные значения в диапазон от -1 до 1
        dropout_mask = np.random.randint(2, size=layer_1.shape) # создаем маску вида array([[0, 1, 0, ... 0, 0, 1]]),
                                                                # где вероятность единиц и нулей по 50% (128,10816)
        layer_1 *= dropout_mask * 2 # применяем маску, но умноженную на 2 
                        # (т.к. сигнал от слоя 1 примерно в 2 раза уменьшается после применения маски, поэтому его нужно усилить)
                        # (128,10816)
        layer_2 = softmax(layer_1.dot(weights_1_2).reshape(-1,num_labels)) # скалярное перемножение layer_1 (128,10816) 
                                            # на веса (10816,10), получаем выходной слой (128,10) с вероятностями каждого лейбла для
                                            # каждого изображения из батча.
        # теперь посчитаем отклонение
        for k in range(batch_size): # итерируемся по примерам в батче, берем каждое изображение по очереди
            labelset = labels[batch_start:batch_end,:] # правильные ответы
            _inc = int(np.argmax(layer_2[k,:]) == np.argmax(labelset[k,:])) # считаем количество True-Positive ответов
            correct_cnt += _inc # учитываем True-Positive ответ (если есть)
                                                    # в общем показателей correct_cnt для j-той итерации (по j-тому батчу)
        layer_2_delta = (layer_2 - labels[batch_start:batch_end,:])/\
                                            (batch_size * layer_2.shape[0]) # считаем отклонение по выходному слою (128,10)
                                            # делим его на на все количество примеров в батче и количество выходных нейронов layer_2
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1) # считаем отклонение по среднему слою:
                                                        # скалярно перемножаем отклонение по выходному слою (128,10)
                                                        # на транспонированную матрицу весов weights_1_2 (10,10816)
                                                        # получаем вектор (128,10816) и не забываем передать градиент
        layer_1_delta *= dropout_mask # при обратном распространении корректировать обнуленные маской нейроны не следует
                                        # поэтому той же маской обнуляем полученные дельты
        weights_1_2 -= alpha * layer_1.T.dot(layer_2_delta) # корректируем веса для выходного слоя: 
                                                            # из матрицы текущих весов (10816,10) вычитаем
                                                            # помноженный на скорость обучения слой 1 транспонированный (10816,128)
                                                            # скалярно помноженный на отклонение по слою 2 (128,10)
                                                            # получаем (10816,10) - такой же формы, как и были наши веса weights_1_2
        # корректируем веса в ядрах
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape) # меняем форму layer_1_delta на форму входного слоя (86528,16) 
        k_update = flattened_input.T.dot(l1d_reshape) # считаем дельту
        kernels -= alpha * k_update # и вычитаем дельту, получаем обновленные веса
        
    test_correct_cnt = 0 # correct_cnt - количество True-Positive ответов

    for i in range(test_images.shape[0]): # бежим по тестовым примерам, тут уже без батчей и корректировки весов разумеется
        layer_0 = test_images[i,:].reshape(1,28,28) # подаем на входной слой изображение (1,28,28)
        sects = []
        for row_start in range(layer_0.shape[1] - kernel_rows + 1): # бежим по строкам от 0 до 28-3 включительно
            for col_start in range(layer_0.shape[2] - kernel_cols + 1): # бежим по столбцам от 0 до 28-3 включительно
                sect = get_image_section(layer_0,
                                row_start, # будет перебираться от 0 до 25 включительно
                                row_start + kernel_rows, # будет перебираться от 3 до 28 включительно
                                col_start, # будет перебираться от 0 до 25 включительно
                                col_start + kernel_cols) # будет перебираться от 3 до 28 включительно
                                #         print(f'row_start = {row_start} col_start = {col_start}')
                sects.append(sect) # всего д.б. 26*26 = 676 массивов размера (128, 1, 3, 3)

        expanded_input = np.concatenate(sects, axis=1) # соединяем массивы горизонтально (присоединяем справа) получаем (128, 676, 3, 3)
        es = expanded_input.shape # (128, 676, 3, 3)
        flattened_input = expanded_input.reshape(es[0]*es[1], -1)
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1)) # 
        layer_2 = layer_1.dot(weights_1_2).reshape(-1,num_labels) # 
        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i,:])) # учитываем True-Positive ответ (если есть)
                                                                            # в общем показателей correct_cnt для j-той итерации
    if j % 1 == 0:
        print("Iter:" + str(j) +\
             " Test_Accuracy: " + str(test_correct_cnt/float(test_images.shape[0])) +\
              " Train_Accuracy: " + str(correct_cnt/float(images.shape[0])) + \
              " time: " + str(time.process_time() - start_time) + " sec")
print("Total time: ", time.process_time() - glob_start_time, " sec")

Iter:0 Test_Accuracy: 0.228 Train_Accuracy: 0.153 time: 13.0625seconds
Iter:1 Test_Accuracy: 0.245 Train_Accuracy: 0.18 time: 13.703125seconds
Iter:2 Test_Accuracy: 0.302 Train_Accuracy: 0.224 time: 13.640625seconds
Iter:3 Test_Accuracy: 0.3735 Train_Accuracy: 0.239 time: 12.71875seconds
Iter:4 Test_Accuracy: 0.456 Train_Accuracy: 0.301 time: 11.734375seconds
Iter:5 Test_Accuracy: 0.5405 Train_Accuracy: 0.359 time: 13.015625seconds
Iter:6 Test_Accuracy: 0.5945 Train_Accuracy: 0.409 time: 12.484375seconds
Iter:7 Test_Accuracy: 0.6255 Train_Accuracy: 0.472 time: 12.578125seconds
Iter:8 Test_Accuracy: 0.6465 Train_Accuracy: 0.529 time: 12.375seconds
Iter:9 Test_Accuracy: 0.6645 Train_Accuracy: 0.595 time: 13.25seconds
Iter:10 Test_Accuracy: 0.676 Train_Accuracy: 0.607 time: 14.234375seconds
Iter:11 Test_Accuracy: 0.6925 Train_Accuracy: 0.64 time: 13.96875seconds
Iter:12 Test_Accuracy: 0.704 Train_Accuracy: 0.682 time: 14.546875seconds
Iter:13 Test_Accuracy: 0.7125 Train_Accuracy: 0.69 tim

Iter:112 Test_Accuracy: 0.8365 Train_Accuracy: 0.905 time: 14.640625seconds
Iter:113 Test_Accuracy: 0.835 Train_Accuracy: 0.895 time: 14.078125seconds
Iter:114 Test_Accuracy: 0.8345 Train_Accuracy: 0.905 time: 14.046875seconds
Iter:115 Test_Accuracy: 0.8345 Train_Accuracy: 0.912 time: 12.796875seconds
Iter:116 Test_Accuracy: 0.8355 Train_Accuracy: 0.902 time: 13.109375seconds
Iter:117 Test_Accuracy: 0.8355 Train_Accuracy: 0.907 time: 12.515625seconds
Iter:118 Test_Accuracy: 0.836 Train_Accuracy: 0.905 time: 13.015625seconds
Iter:119 Test_Accuracy: 0.836 Train_Accuracy: 0.91 time: 12.84375seconds
Iter:120 Test_Accuracy: 0.837 Train_Accuracy: 0.904 time: 13.015625seconds
Iter:121 Test_Accuracy: 0.838 Train_Accuracy: 0.912 time: 12.546875seconds
Iter:122 Test_Accuracy: 0.836 Train_Accuracy: 0.895 time: 12.453125seconds
Iter:123 Test_Accuracy: 0.8365 Train_Accuracy: 0.9 time: 13.21875seconds
Iter:124 Test_Accuracy: 0.837 Train_Accuracy: 0.909 time: 12.84375seconds
Iter:125 Test_Accuracy: 0

Iter:222 Test_Accuracy: 0.837 Train_Accuracy: 0.938 time: 10.1875seconds
Iter:223 Test_Accuracy: 0.835 Train_Accuracy: 0.934 time: 10.28125seconds
Iter:224 Test_Accuracy: 0.835 Train_Accuracy: 0.921 time: 9.921875seconds
Iter:225 Test_Accuracy: 0.834 Train_Accuracy: 0.946 time: 9.84375seconds
Iter:226 Test_Accuracy: 0.835 Train_Accuracy: 0.933 time: 10.03125seconds
Iter:227 Test_Accuracy: 0.835 Train_Accuracy: 0.937 time: 9.703125seconds
Iter:228 Test_Accuracy: 0.8325 Train_Accuracy: 0.936 time: 10.09375seconds
Iter:229 Test_Accuracy: 0.8325 Train_Accuracy: 0.935 time: 10.046875seconds
Iter:230 Test_Accuracy: 0.8365 Train_Accuracy: 0.938 time: 10.828125seconds
Iter:231 Test_Accuracy: 0.8355 Train_Accuracy: 0.927 time: 10.0625seconds
Iter:232 Test_Accuracy: 0.835 Train_Accuracy: 0.929 time: 10.328125seconds
Iter:233 Test_Accuracy: 0.8355 Train_Accuracy: 0.941 time: 10.03125seconds
Iter:234 Test_Accuracy: 0.833 Train_Accuracy: 0.94 time: 9.96875seconds
Iter:235 Test_Accuracy: 0.8355 Trai